# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lajas,18.0500,-67.0593,26.34,58,12,0.45,PR,1699423708
1,1,ancud,-41.8697,-73.8203,8.62,89,74,1.21,CL,1699423709
2,2,wailua homesteads,22.0669,-159.3780,25.44,85,100,5.14,US,1699423709
3,3,keflavik,64.0049,-22.5624,6.24,56,0,7.72,IS,1699423709
4,4,port alfred,-33.5906,26.8910,18.43,76,74,1.34,ZA,1699423709


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
       "Lng",
       "Lat",
       geo = True,
       tiles = "OSM",
       frame_width = 700,
       frame_height = 500,
       size = "Humidity",
       scale = 1.0,
       color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_condition_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &  # Max temperature less than 27°C
    (city_data_df["Max Temp"] > 21) &  # Max temperature greater than 21°C
    (city_data_df["Humidity"] < 70) &      # Humidity less than 70%
    (city_data_df["Cloudiness"] < 15) &    # Cloudiness less than 15%
    (city_data_df["Wind Speed"] < 4.5)       # Wind speed less than 4.5 m/s
]

# Drop any rows with null values
ideal_condition_df = ideal_condition_df.dropna()

# Display sample data
ideal_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lajas,18.0500,-67.0593,26.34,58,12,0.45,PR,1699423708
18,18,tura,25.5198,90.2201,26.12,56,9,1.19,IN,1699423713
92,92,mulayjah,27.2710,48.4242,22.90,29,0,3.91,SA,1699423732
130,130,al burayqah,30.4062,19.5739,21.51,59,2,2.81,LY,1699423742
248,248,khanbari,27.3747,87.2039,22.69,55,10,2.54,NP,1699423774
340,340,ghanzi,-21.5667,21.7833,23.44,29,0,4.32,BW,1699423799
403,403,dayr hafir,36.1569,37.7078,21.13,36,0,1.65,SY,1699423830
405,405,mahanje,-9.9333,35.3333,26.09,52,11,2.31,TZ,1699423831
507,507,kayes,14.0000,-11.0000,24.37,26,7,1.24,ML,1699423862


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_condition_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lajas,PR,18.0500,-67.0593,58,
18,tura,IN,25.5198,90.2201,56,
92,mulayjah,SA,27.2710,48.4242,29,
130,al burayqah,LY,30.4062,19.5739,59,
248,khanbari,NP,27.3747,87.2039,55,
340,ghanzi,BW,-21.5667,21.7833,29,
403,dayr hafir,SY,36.1569,37.7078,36,
405,mahanje,TZ,-9.9333,35.3333,52,
507,kayes,ML,14.0000,-11.0000,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation",
    "radius": radius,
    "limit": 1,
    "apiKey": geoapify_key,  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lajas - nearest hotel: Hotel y Parador Oasis
tura - nearest hotel: No hotel found
mulayjah - nearest hotel: No hotel found
al burayqah - nearest hotel: فندق ماس ليبيا
khanbari - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
dayr hafir - nearest hotel: No hotel found
mahanje - nearest hotel: No hotel found
kayes - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,lajas,PR,18.0500,-67.0593,58,Hotel y Parador Oasis
18,tura,IN,25.5198,90.2201,56,No hotel found
92,mulayjah,SA,27.2710,48.4242,29,No hotel found
130,al burayqah,LY,30.4062,19.5739,59,فندق ماس ليبيا
248,khanbari,NP,27.3747,87.2039,55,No hotel found
340,ghanzi,BW,-21.5667,21.7833,29,Ghanzi farmhouse
403,dayr hafir,SY,36.1569,37.7078,36,No hotel found
405,mahanje,TZ,-9.9333,35.3333,52,No hotel found
507,kayes,ML,14.0000,-11.0000,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
       "Lng",
       "Lat",
       geo = True,
       tiles = "OSM",
       frame_width = 700,
       frame_height = 500,
       size = "Humidity", 
       scale = 1.0,
       color = "City",
       hover_cols=["City", "Hotel Name", "Country"]  # Add hotel name and country to the hover message
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)